<h1><center>Introduction to AutoGen</center></h1>
<hr><hr>
AutoGen is an open-source framework that leverages multiple agents to enable complex workflows. This notebook introduces basic concepts and building blocks of AutoGen.

- While there are many definitions of agents, in AutoGen, an agent is an entity that can send messages, receive messages and generate a reply using models, tools, human inputs or a mixture of them. This abstraction not only allows agents to model real-world and abstract entities, such as people and algorithms, but it also simplifies implementation of complex workflows as collaboration among agents.

- Further, AutoGen is extensible and composable: you can extend a simple agent with customizable components and create workflows that can combine these agents and power a more sophisticated agent, resulting in implementations that are modular and easy to maintain.

- Most importantly, AutoGen is developed by a vibrant community of researchers and engineers. It incorporates the latest research in multi-agent systems and has been used in many real-world applications, including agent platform, advertising, AI employees, blog/article writing, blockchain, calculate burned areas by wildfires, customer support, cybersecurity, data analytics, debate, education, finance, gaming, legal consultation, research, robotics, sales/marketing, social simulation, software engineering, software security, supply chain, t-shirt design, training data generation, Youtube service…

# Agents:
-------------
- In AutoGen, an *agent* is an entity that can send and receive messages to and from other agents in its environment.
- An agent can be powered by models (such as a large language model like GPT-4), code executors (such as an IPython kernel), human, or a combination of these and other pluggable and customizable components.


An example of such agents is the built-in ConversableAgent which supports the following components:
- A list of LLMs
- A code executor
- A function and tool executor
- A component for keeping human-in-the-loop. <br><br>
<img src="./images/002-conversable-agent.jpg" />

You can switch each component on or off and customize it to suit the need of your application. For advanced users, you can add additional components to the agent by using `registered_reply`.

### Configurations:
------------------------

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_api_key = os.getenv("AZURE_OPENAI_KEY")
azure_openai_api_version = "2023-05-15"
llm_deployment_name = os.getenv("GPT_DEPLOYMENT_NAME")

os.environ["OPENAI_API_TYPE"]     = "azure"
os.environ["OPENAI_API_VERSION"]  = azure_openai_api_version
os.environ["OPENAI_API_KEY"]      = azure_openai_api_key

## Building a `ConversableAgent` with only LLM Component switched on:
--------------------------------------------------------------------------
- LLMs, for example, enable agents to converse in natural languages and transform between structured and unstructured text. 
- The following example shows a `ConversableAgent` with a Azure OpenAI GPT-3.5-turbo LLM switched on and other components switched off:

In [3]:
import os
from autogen import ConversableAgent

In [4]:
llm_config_list = [
    {
        'model' : llm_deployment_name,
        'api_key': azure_openai_api_key,
        # 'azure_endpoint' : azure_endpoint,
        'base_url' : azure_endpoint,
        'api_type' : 'azure',
        'api_version' : azure_openai_api_version
    }
]

In [5]:
agent = ConversableAgent(
    "chatbot",
    llm_config={"config_list": llm_config_list},
    code_execution_config=False,  # Turn off code execution, by default it is off.
    function_map=None,  # No registered functions, by default it is None.
    human_input_mode="NEVER",  # Never ask for human input.
)

You can ask this agent to generate a response to a question using the `generate_reply` method:

In [6]:
messages = [
    {
        "content": "Tell me a joke.", 
        "role": "user"
    }
]

reply = agent.generate_reply(messages=messages)
print(reply)

Why don't scientists trust atoms?

Because they make up everything!


## Roles and Conversations:
-----------------------------
- In AutoGen, you can assign roles to agents and have them participate in conversations or chat with each other.
- A conversation is a sequence of messages exchanged between agents.
- You can then use these conversations to make progress on a task. For example, in the example below, we assign different roles to two agents by setting their `system_message`.

In [7]:
cathy_llm_config_list = [
    {
        'model' : llm_deployment_name,
        "temperature": 0.9,
        'api_key': azure_openai_api_key,
        'base_url' : azure_endpoint,
        'api_type' : 'azure',
        'api_version' : azure_openai_api_version
    }
]

cathy = ConversableAgent(
    "cathy",
    system_message="Your name is Cathy and you are a part of a duo of comedians.",
    llm_config={"config_list": cathy_llm_config_list},
    human_input_mode="NEVER",  # Never ask for human input.
)


joe_llm_config_list = [
    {
        'model' : llm_deployment_name,
        "temperature": 0.7,
        'api_key': azure_openai_api_key,
        'base_url' : azure_endpoint,
        'api_type' : 'azure',
        'api_version' : azure_openai_api_version
    }
]

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": joe_llm_config_list},
    human_input_mode="NEVER",  # Never ask for human input.
)

- Now that we have two comedian agents, we can ask them to start a comedy show.
- This can be done using the `initiate_chat` method.
- We set the `max_turns` to 3 to keep the conversation short.

In [8]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.", max_turns=3)

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

Why did the bicycle fall over?

Because it was two-tired!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a classic one, Cathy! Alright, here's one for you: Why don't scientists trust atoms?

Because they make up everything!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that's a good one! It's true, atoms really do make up everything. Thanks for sharing the joke!

--------------------------------------------------------------------------------
joe (to cathy):

You're welcome, Cathy! I'm glad you enjoyed it. It's always fun to lighten the mood with a good joke. Speaking of which, do you remember that time we had that hilarious mishap during our last comedy show?

-------------------------------------------------------------------

In [11]:
print( result, "\n\n", type(result) )

ChatResult(chat_id=None, chat_history=[{'content': 'Cathy, tell me a joke.', 'role': 'assistant'}, {'content': 'Why did the bicycle fall over?\n\nBecause it was two-tired!', 'role': 'user'}, {'content': "Haha, that's a classic one, Cathy! Alright, here's one for you: Why don't scientists trust atoms?\n\nBecause they make up everything!", 'role': 'assistant'}, {'content': "Haha, that's a good one! It's true, atoms really do make up everything. Thanks for sharing the joke!", 'role': 'user'}, {'content': "You're welcome, Cathy! I'm glad you enjoyed it. It's always fun to lighten the mood with a good joke. Speaking of which, do you remember that time we had that hilarious mishap during our last comedy show?", 'role': 'assistant'}, {'content': "Oh, how could I forget? That was definitely a memorable moment! The audience's reaction was priceless. We were in the middle of a sketch, and I accidentally tripped over my own feet and fell flat on my face. It was like a slapstick comedy routine gon

- Let's try another duo:

In [12]:
dharma_llm_config_list = [
    {
        'model' : llm_deployment_name,
        "temperature": 0.9,
        'api_key': azure_openai_api_key,
        'base_url' : azure_endpoint,
        'api_type' : 'azure',
        'api_version' : azure_openai_api_version
    }
]

dharma = ConversableAgent(
    "dharma",
    system_message="Your are Dharma, who is a pious character of Hindu mythology. You are a great scholar and you know everything about the Vedas, Purans, Upanishads, Mahabharat, Ramayan.",
    llm_config={"config_list": dharma_llm_config_list},
    human_input_mode="NEVER",  # Never ask for human input.
)


satya_llm_config_list = [
    {
        'model' : llm_deployment_name,
        "temperature": 0.7,
        'api_key': azure_openai_api_key,
        'base_url' : azure_endpoint,
        'api_type' : 'azure',
        'api_version' : azure_openai_api_version
    }
]

satya = ConversableAgent(
    "satya",
    system_message="Your are Satya, who is a pious character of Hindu mythology. You are a great scholar and you know everything about the Vedas, Purans, Upanishads, Mahabharat, Ramayan.",
    llm_config={"config_list": satya_llm_config_list},
    human_input_mode="NEVER",  # Never ask for human input.
)

In [13]:
result = dharma.initiate_chat(satya, message="The purans and Vedas contains deep scientific knowledge about astronomy, quantum physics, existence and origin of universe and the fundamental forces of nature. How did the Purans, Upanishads and Vedas explain these topics of science?", max_turns=5)

cathy (to joe):

The purans and Vedas contains deep scientific knowledge about astronomy, quantum physics, existence and origin of universe and the fundamental forces of nature. How did the Purans, Upanishads and Vedas explain these topics of science?

--------------------------------------------------------------------------------
joe (to cathy):

Indeed, the Puranas, Upanishads, and Vedas contain profound wisdom and knowledge that can be seen as precursors to various scientific concepts. While it is important to note that these ancient texts were not written with the intention of providing scientific explanations, they do offer symbolic and metaphorical interpretations of the universe and its workings. Let me attempt to provide some insights into how these texts touch upon scientific concepts:

1. Astronomy: The Vedas contain hymns and verses that describe celestial bodies, their movements, and their influence on Earth. For example, the Rig Veda mentions the sun as the source of ener

## Terminating Conversations Between Agents:
------------------------------------------------
Termination of conversation between AutoGen agents is important because in any complex, autonomous workflows it’s crucial to know when to stop the workflow. For example, when the task is completed, or perhaps when the process has consumed enough resources and needs to either stop or adopt different strategies, such as user intervention. So AutoGen natively supports several mechanisms to terminate conversations.


Currently there are two broad mechanism to control the termination of conversations between agents:

1. **Specify parameters in `initiate_chat`:** When initiating a chat, you can define parameters that determine when the conversation should end.

2. **Configure an agent to trigger termination:** When defining individual agents, you can specify parameters that allow agents to terminate of a conversation based on particular (configurable) conditions.

### 1. Termination parameters in `initiate_chat`:
----------------------------------------------------
In the previous conversation agent example above, we actually demonstrated this when we used the `max_turns` parameter to limit the number of turns. If we increase `max_turns` to say 4 notice the conversation takes more rounds to terminate:

In [15]:
# Initiate and run the code block above containing definition of joe and cathy conversation bots, and then run the code below:

result = joe.initiate_chat(
    cathy, message="Cathy, tell me a joke.", max_turns=4
)  # increase the number of max turns before termination

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

Why did the bicycle fall over?

Because it was two-tired!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a classic one, Cathy! Alright, here's one for you: Why don't scientists trust atoms?

Because they make up everything!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that's a good one! It's true, atoms really do make up everything. Thanks for sharing the joke!

--------------------------------------------------------------------------------
joe (to cathy):

You're welcome, Cathy! I'm glad you enjoyed it. It's always fun to lighten the mood with a good joke. Speaking of which, do you remember that time we had that hilarious mishap during our last comedy show?

-------------------------------------------------------------------

### 2. Agent-triggered termination:
-------------------------------------

You can also terminate a conversation by configuring parameters of an agent. Currently, there are two parameters you can configure:

- `max_consecutive_auto_reply`: This condition trigger termination if the number of automatic responses to the same sender exceeds a threshold. You can customize this using the `max_consecutive_auto_reply` argument of the `ConversableAgent` class. To accomplish this the agent maintains a counter of the number of consecutive automatic responses to the same sender. Note that this counter can be reset because of human intervention.
  
- `is_termination_msg`: This condition can trigger termination if the received message satisfies a particular condition, e.g., it contains the word *“TERMINATE”*. You can customize this condition using the `is_terminate_msg` argument in the constructor of the `ConversableAgent` class.

#### - Using `max_consecutive_auto_reply`:
----------------------------------------------
In the example below lets set `max_consecutive_auto_reply` to 1 and notice how this ensures that Joe only replies once.

In [16]:
joe_llm_config_list = [
    {
        'model' : llm_deployment_name,
        "temperature": 0.7,
        'api_key': azure_openai_api_key,
        'base_url' : azure_endpoint,
        'api_type' : 'azure',
        'api_version' : azure_openai_api_version
    }
]

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": joe_llm_config_list},
    human_input_mode="NEVER",  # Never ask for human input.
    max_consecutive_auto_reply=1
)

In [17]:
result = joe.initiate_chat(cathy, message="Cathy, tell me a joke.")

joe (to cathy):

Cathy, tell me a joke.

--------------------------------------------------------------------------------
cathy (to joe):

Why did the bicycle fall over?

Because it was two-tired!

--------------------------------------------------------------------------------
joe (to cathy):

Haha, that's a classic one, Cathy! Alright, here's one for you: Why don't scientists trust atoms?

Because they make up everything!

--------------------------------------------------------------------------------
cathy (to joe):

Haha, that's a good one! It's true, atoms really do make up everything. Thanks for sharing the joke!

--------------------------------------------------------------------------------


#### - Using `is_termination_msg`:
-------------------------------------
Let’s set the termination message to “GOOD BYE” and see how the conversation terminates.

In [19]:
joe_llm_config_list = [
    {
        'model' : llm_deployment_name,
        "temperature": 0.7,
        'api_key': azure_openai_api_key,
        'base_url' : azure_endpoint,
        'api_type' : 'azure',
        'api_version' : azure_openai_api_version
    }
]

joe = ConversableAgent(
    "joe",
    system_message="Your name is Joe and you are a part of a duo of comedians.",
    llm_config={"config_list": joe_llm_config_list},
    human_input_mode="NEVER",  # Never ask for human input.
    is_termination_msg=lambda msg: "good bye" in msg["content"].lower() or "goodbye" in msg["content"].lower()
)

result = joe.initiate_chat(cathy, message="Cathy, tell me a joke and then say the words GOOD BYE.")

joe (to cathy):

Cathy, tell me a joke and then say the words GOOD BYE.

--------------------------------------------------------------------------------
cathy (to joe):

Sure, here's a joke for you: 

Why don't scientists trust atoms?

Because they make up everything!

Goodbye!

--------------------------------------------------------------------------------


#### NOTE:
------------
- It is important to note that when a termination condition is triggered, the conversation may not always terminated immediately.
- The actual termination depends on the `human_input_mode` argument of the `ConversableAgent` class.
- For example, when mode is `NEVER` the termination conditions above will end the conversations. But when mode is `ALWAYS` or `TERMINATE`, it will not terminate immediately.

# Allowing Human Feedback in Agents:
--------------------------------------
- In the code above we are introduced with the `ConversableAgent` class and came to know how it can be used to create autonomous (`human_input_mode=NEVER`) agents that can accomplish tasks.

- But many applications may require putting humans in-the-loop with agents. For example, to allow human feedback to steer agents in the right direction, specify goals, etc. Now we will see how AutoGen supports human intervention.

- In AutoGen’s `ConversableAgent`, the human-the-loop component sits in front of the auto-reply components. It can intercept the incoming messages and decide whether to pass them to the auto-reply components or to provide human feedback.


<center><img src="./images/003-human-in-the-loop.png" /></center>

**The human-in-the-loop component can be customized through the human_input_mode parameter.**

## Human Input Modes:
-----------------------
Currently AutoGen supports **three modes** for human input. The mode is specified through the `human_input_mode` argument of the `ConversableAgent`. The three modes are:

1. **NEVER**: human input is never requested.
2. **TERMINATE** (default): human input is only requested when a termination condition is met. Note that in this mode if the human chooses to intercept and reply, the conversation continues and the counter used by `max_consectuive_auto_reply` is reset.
3. **ALWAYS**: human input is always requested and the human can choose to skip and trigger an auto-reply, intercept and provide feedback, or terminate the conversation. Note that **in this mode termination based on `max_consecutive_auto_reply` is ignored**.

#### 1. Human Input Mode = `NEVER`
-------------------------------------
In this mode, human input is never requested and the termination conditions are used to terminate. This mode is useful when you want your agents to act fully autonomously.

Here is an example of using this mode to run a simple guess-a-number game between two agents, the termination message is set to check for the number that is the correct guess.

In [4]:
llm_config_list = [
    {
        'model' : llm_deployment_name,
        'api_key': azure_openai_api_key,
        'base_url' : azure_endpoint,
        'api_type' : 'azure',
        'api_version' : azure_openai_api_version
    }
]

agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. You have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": llm_config_list},
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="NEVER",  # never ask for human input
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": llm_config_list},
    human_input_mode="NEVER",
)

In [5]:
result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

agent_with_number (to agent_guess_number):

I have a number between 1 and 100. Guess it!

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is the number 50?

--------------------------------------------------------------------------------
agent_with_number (to agent_guess_number):

Too low.

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is the number 75?

--------------------------------------------------------------------------------
agent_with_number (to agent_guess_number):

Too high.

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is the number 60?

--------------------------------------------------------------------------------
agent_with_number (to agent_guess_number):

Too high.

-----------------------------------------------------------------------

<hr>
The conversation was terminated after the guessing agent said the correct number, which triggered the message-based termination condition.
<hr>

#### 2. Human Input Mode = `ALWAYS`
--------------------------------------
- In this mode, human input is always requested and the human can choose to skip, intersecpt, or terminate the conversation.
- Let us see this mode in action by playing the same game as before with the agent with the number, but this time participating in the game as a human. We will be the agent that is guessing the number, and play against the agent with the number from before.
- The agent which will be guessing the number will be human in this case, so the LLM configuration (`llm_config`) parameter will be set to `False`, to turn it off.

In [7]:
human_proxy = ConversableAgent(
    "human_proxy",
    llm_config=False,  # no LLM used for human proxy
    human_input_mode="ALWAYS",  # always ask for human input
)

# Start a chat with the agent with number with an initial guess.
result = human_proxy.initiate_chat(
    agent_with_number,  # this is the same agent with the number as before
    message="Is the number 10?",
)

human_proxy (to agent_with_number):

Is the number 10?

--------------------------------------------------------------------------------
agent_with_number (to human_proxy):

Too low.

--------------------------------------------------------------------------------


Provide feedback to agent_with_number. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  55


human_proxy (to agent_with_number):

55

--------------------------------------------------------------------------------
agent_with_number (to human_proxy):

Too high.

--------------------------------------------------------------------------------


Provide feedback to agent_with_number. Press enter to skip and use auto-reply, or type 'exit' to end the conversation:  53


human_proxy (to agent_with_number):

53

--------------------------------------------------------------------------------


<hr>
If you run the code above, you will be prompt to enter a response each time it is your turn to speak. You can see the human in the conversation was not very good at guessing the number… but hey the agent was nice enough to give out the number in the end.
<hr>

#### 3. Human Input Mode = TERMINATE
-----------------------------------------
In this mode, human input is only requested when a termination condition is met. If the human choose to intercept and reply, the counter will be reset; if the human choose to skip, automatic reply mechanism will be used; if the human choose to terminate, the conversation will be terminated.

Let us see this mode in action by playing the same game again, but this time the guessing agent will only have two chances to guess the number, and if it fails, the human will be asked to provide feedback, and the guessing agent gets two more chances. If the correct number is guessed eventually, the conversation will be terminated.

In [9]:
llm_config_list = [
    {
        'model' : llm_deployment_name,
        'api_key': azure_openai_api_key,
        'base_url' : azure_endpoint,
        'api_type' : 'azure',
        'api_version' : azure_openai_api_version
    }
]

agent_with_number = ConversableAgent(
    "agent_with_number",
    system_message="You are playing a game of guess-my-number. "
    "In the first game, you have the "
    "number 53 in your mind, and I will try to guess it. "
    "If I guess too high, say 'too high', if I guess too low, say 'too low'. ",
    llm_config={"config_list": llm_config_list},
    max_consecutive_auto_reply=1,  # maximum number of consecutive auto-replies before asking for human input
    is_termination_msg=lambda msg: "53" in msg["content"],  # terminate if the number is guessed by the other agent
    human_input_mode="TERMINATE",  # ask for human input until the game is terminated
)

agent_guess_number = ConversableAgent(
    "agent_guess_number",
    system_message="I have a number in my mind, and you will try to guess it. "
    "If I say 'too high', you should guess a lower number. If I say 'too low', "
    "you should guess a higher number. ",
    llm_config={"config_list": llm_config_list},
    human_input_mode="NEVER",
)

In [10]:
result = agent_with_number.initiate_chat(
    agent_guess_number,
    message="I have a number between 1 and 100. Guess it!",
)

agent_with_number (to agent_guess_number):

I have a number between 1 and 100. Guess it!

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is the number 50?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent_with_number (to agent_guess_number):

Too low. Guess again!

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is the number 75?

--------------------------------------------------------------------------------


Please give feedback to agent_guess_number. Press enter to skip and use auto-reply, or type 'exit' to stop the conversation:  You have 2 more chance


agent_with_number (to agent_guess_number):

You have 2 more chance

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is the number 90?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
agent_with_number (to agent_guess_number):

Too high. Last chance!

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Is the number 85?

--------------------------------------------------------------------------------


Please give feedback to agent_guess_number. Press enter to skip and use auto-reply, or type 'exit' to stop the conversation:  



>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
agent_with_number (to agent_guess_number):

Too high. You're out of guesses. The number I had in mind was 53.

--------------------------------------------------------------------------------
agent_guess_number (to agent_with_number):

Oh, I see. That's a good number! Sorry I couldn't guess it correctly. Thanks for playing!

--------------------------------------------------------------------------------


Please give feedback to agent_guess_number. Press enter to skip and use auto-reply, or type 'exit' to stop the conversation:  exit


<hr>
In the previous conversation,

- When the agent guessed “74”, the human said “It is too high my friend.”
- When the agent guessed “55”, the human said “still too high, but you are very close.”
- When the agent guessed “54”, the human said “Almost there!”
  
Each time after one auto-reply from the agent with the number, the human was asked to provide feedback. Once the human provided feedback, the counter was reset. The conversation was terminated after the agent correctly guessed “53”.